## 1. Start

In [1]:
import pandas as pd
import numpy as np
import csv
assert csv  # silence pyflakes
import xlsxwriter
assert xlsxwriter  # silence pyflakes
import xlrd
assert xlrd  # silence pyflakes
import math
assert math  # silence pyflakes
import nbimporter
import traceback
from pathlib import Path
import os

In [2]:
try:
    libraries_path = Path(__file__).resolve()
except NameError:
    libraries_path = Path(os.getcwd()) / "Libraries.ipynb"
    if not libraries_path.exists():
        libraries_path = Path(os.getcwd())

Libraries_dir = libraries_path.parent

print(f"📁 Libraries.ipynb — directory impostata a: {Libraries_dir}")


📁 Libraries.ipynb — directory impostata a: C:\Users\RELab\OneDrive - Politecnico di Milano\grupporelab\02_Progetti\AREA LCA\Kevin\PhD\aa20232024\03_Sviluppo\Nuovo modello\FINALE\bw


# 2. Files reading

In [3]:
# ✅ 

base_path = Libraries_dir
root_path = os.path.join(base_path, "Energy_systems")
    
EndOfLifeInputParameters_path = base_path / 'EndOfLifeInputParameters.xlsx'
generatorsConfig_path = base_path / 'DataInput.xlsm'
energysystem_path = base_path / 'DataInput.xlsm'

EndOfLifeInputParameters = pd.read_excel(EndOfLifeInputParameters_path, sheet_name='EndOfLifeInputParameters')
generatorsConfig = pd.read_excel(generatorsConfig_path, sheet_name='generatorsConfig')

In [4]:
boiler_path = os.path.join(root_path, 'Boiler.ipynb')
Biomassboiler_path = os.path.join(root_path, 'BiomassBoiler.ipynb')
Fueloilboiler_path = os.path.join(root_path, 'FuelOilBoiler.ipynb')
EHP_path = os.path.join(root_path, 'ElectricHeatPump.ipynb')
GEHP_path = os.path.join(root_path, 'GasEngineHeatPump.ipynb')
GAHP_path = os.path.join(root_path, 'GasAbsorptionHeatPump.ipynb')
Geothermal_path = os.path.join(root_path, 'GeothermalPlant.ipynb')
RICE_path = os.path.join(root_path, 'CombinedHeatPower_InternalCombustionEngine.ipynb')
ORC_path = os.path.join(root_path, 'CombinedHeatPower_OrganicRankineCycle.ipynb')
WtE_path = os.path.join(root_path, 'CombinedHeatPower_WasteToEnergy.ipynb')
CombinedCycle_path = os.path.join(root_path, 'CombinedHeatPower_CombinedCycle.ipynb')
GasTurbine_path = os.path.join(root_path, 'CombinedHeatPower_Turbogas.ipynb')
HeatRecovery_path = os.path.join(root_path, 'HeatRecovery.ipynb')
SolarThermal_path = os.path.join(root_path, 'SolarThermal.ipynb')
ThermalConcreteStorage_path = os.path.join(root_path, 'ThermalConcreteStorage.ipynb')
ThermalSteelStorage_path = os.path.join(root_path, 'ThermalSteelStorage.ipynb')
AbsorptionChiller_path = os.path.join(root_path, 'CoolingAbsorptionChiller.ipynb')
CompressionChiller_path = os.path.join(root_path, 'CoolingCompressionChiller.ipynb')
DryCooler_path = os.path.join(root_path, 'CoolingDryCooler.ipynb')
CoolingTower_path = os.path.join(root_path, 'CoolingEvaporativeTower.ipynb')
Auxiliaries_path = os.path.join(root_path, 'Auxiliaries.ipynb')

# 3. General information

In [5]:
Year = generatorsConfig.loc[generatorsConfig['Information'] == 'Reference scenario year', 'Data'].values[0]
DHTypology = generatorsConfig.loc[generatorsConfig['Information'] == 'District Heating and Cooling typology', 'Data'].values[0]
Aim = generatorsConfig.loc[generatorsConfig['Information'] == 'Aim of the LCA analysis', 'Data'].values[0]

RSP = generatorsConfig.loc[generatorsConfig['Information'] == 'Reference Study Period (RSP) [Year]', 'Data'].values[0]
ReqSL = generatorsConfig.loc[generatorsConfig['Information'] == 'Required Service Life (ReqSL) [Year]', 'Data'].values[0]
Losses = generatorsConfig.loc[generatorsConfig['Information'] == 'Distribution losses', 'Data'].values[0]

Multifunctionality = generatorsConfig.loc[generatorsConfig['Information'] == 'Life Cycle Assessment Multifunctionality approach', 'Data'].values[0]
ExternalTemperature = generatorsConfig.loc[generatorsConfig['Information'] == 'Reference External Temperature [°C]', 'Data'].values[0]
SupplyTemperaturePrimaryCircuit = generatorsConfig.loc[generatorsConfig['Information'] == 'Supply primary circuit Temperature [°C]', 'Data'].values[0]
ReturnTemperaturePrimaryCircuit = generatorsConfig.loc[generatorsConfig['Information'] == 'Return primary circuit Temperature [°C]', 'Data'].values[0]

TotalLenght = generatorsConfig.loc[generatorsConfig['Information'] == 'Length pipes [km]', 'Data'].values[0]
PipeTechnology = generatorsConfig.loc[generatorsConfig['Information'] == 'Pipe typology', 'Data'].values[0]
Diameter = generatorsConfig.loc[generatorsConfig['Information'] == 'Diameter of the pipes [m]', 'Data'].values[0]
NumberUsersHeat = generatorsConfig.loc[generatorsConfig['Information'] == 'Number of users', 'Data'].values[0]
NumberUsersACS = generatorsConfig.loc[generatorsConfig['Information'] == 'Number of users Domestic Hot Water', 'Data'].values[0]
TerminalTypology = generatorsConfig.loc[generatorsConfig['Information'] == 'Terminal typology', 'Data'].values[0]
TotalEnergyErogated = generatorsConfig.loc[generatorsConfig['Information'] == 'Total thermal energy supplied to the end users [MWh]', 'Data'].values[0]

Grid = generatorsConfig.loc[generatorsConfig['Information'] == 'National Grid', 'Data'].values[0]  # Percentage of EE from grid consumed
RECSPV = generatorsConfig.loc[generatorsConfig['Information'] == 'Renewable Energy Certificates from Photovoltaic electricity production', 'Data'].values[0]  # Percentage of RECs related to the electricity consumed
RECSHydro = generatorsConfig.loc[generatorsConfig['Information'] == 'Renewable Energy Certificates from Hydroelectric electricity production', 'Data'].values[0]  # Percentage of RECs related to the electricity consumed
RECSEolic = generatorsConfig.loc[generatorsConfig['Information'] == 'Renewable Energy Certificates from Eolic electricity production', 'Data'].values[0]  # Percentage of RECs related to the electricity consumed
PV = generatorsConfig.loc[generatorsConfig['Information'] == 'Photovoltaic system', 'Data'].values[0]  # Percentage of PV related to the electricity consumed

# 4. Coefficients for scenarios

In [6]:
if (Year == 2025): #blend
    A = 1
    B = 0
    C = 0
elif (Year == 2030): #blend
    A = 0.90
    B = 0.084
    C = 0.014
elif (Year == 2035): #blend
    A = 0.795
    B = 0.189
    C = 0.016
elif (Year == 2040): #blend
    A = 0.688
    B = 0.294
    C = 0.018
elif (Year == 2045): #blend
    A = 0.582
    B = 0.399
    C = 0.020

elif (Year == 2050): #blend
    A = 0
    B = 0.80
    C = 0.20
else:
    print ("index out of range")

# 5. End-of-Life parameters import

In [7]:
#End-of-life parameters
LandfillMetals2 = (EndOfLifeInputParameters.loc[EndOfLifeInputParameters['Parameter'] == 'Share of metals (e.g low-alloyed steel) sent to recycling at end of life', 'value'].values[0])
LandfillMetals = (EndOfLifeInputParameters.loc[EndOfLifeInputParameters['Parameter'] == 'Share of metals  (e.g low-alloyed steel) sent to landfill at end of life', 'value'].values[0])
LandfillElastomer = (EndOfLifeInputParameters.loc[EndOfLifeInputParameters['Parameter'] == 'Share of elastomers sent to landfill at end of life', 'value'].values[0])
LandfillPVC = (EndOfLifeInputParameters.loc[EndOfLifeInputParameters['Parameter'] == 'Share of PVC sent to landfill at end of life', 'value'].values[0])
LandfillPEHD = (EndOfLifeInputParameters.loc[EndOfLifeInputParameters['Parameter'] == 'Share of high-density polyethylene (HDPE) sent to landfill at end of life', 'value'].values[0])
LandfillRefrigerant = (EndOfLifeInputParameters.loc[EndOfLifeInputParameters['Parameter'] == 'Share of refrigerant gases sent to landfill at end of life', 'value'].values[0])
LandfillElectronics = (EndOfLifeInputParameters.loc[EndOfLifeInputParameters['Parameter'] == 'Share of electronic waste sent to landfill at end of life', 'value'].values[0])
LandfillClaybrick = (EndOfLifeInputParameters.loc[EndOfLifeInputParameters['Parameter'] == 'Share of clay bricks sent to landfill at end of life', 'value'].values[0])
LandfillConcrete = (EndOfLifeInputParameters.loc[EndOfLifeInputParameters['Parameter'] == 'Share of concrete sent to landfill at end of life', 'value'].values[0])
LandfillGlass = (EndOfLifeInputParameters.loc[EndOfLifeInputParameters['Parameter'] == 'Share of glass sent to landfill at end of life', 'value'].values[0])
LandfillMineralWool = (EndOfLifeInputParameters.loc[EndOfLifeInputParameters['Parameter'] == 'Share of mineral wool sent to landfill at end of life', 'value'].values[0])
LandfillBrass = (EndOfLifeInputParameters.loc[EndOfLifeInputParameters['Parameter'] == 'Share of brass sent to landfill at end of life', 'value'].values[0])
LandfillCardboard = (EndOfLifeInputParameters.loc[EndOfLifeInputParameters['Parameter'] == 'Share of cardboard sent to landfill at end of life', 'value'].values[0])
LandfillInert = (EndOfLifeInputParameters.loc[EndOfLifeInputParameters['Parameter'] == 'Share of inert materials sent to landfill at end of life', 'value'].values[0])
LandfillPUfoam = (EndOfLifeInputParameters.loc[EndOfLifeInputParameters['Parameter'] == 'Share of polyurethane foam sent to landfill at end of life', 'value'].values[0])
LandfillPP = (EndOfLifeInputParameters.loc[EndOfLifeInputParameters['Parameter'] == 'Share of polypropylene (PP) sent to landfill at end of life', 'value'].values[0])

IncinerationElastomer = (EndOfLifeInputParameters.loc[EndOfLifeInputParameters['Parameter'] == 'Share of elastomers sent to incineration at end of life', 'value'].values[0])
IncinerationPVC = (EndOfLifeInputParameters.loc[EndOfLifeInputParameters['Parameter'] == 'Share of PVC sent to incineration at end of life', 'value'].values[0])
IncinerationElectronics = (EndOfLifeInputParameters.loc[EndOfLifeInputParameters['Parameter'] == 'Share of electronic waste sent to incineration at end of life', 'value'].values[0])
IncinerationLubricatingOil = (EndOfLifeInputParameters.loc[EndOfLifeInputParameters['Parameter'] == 'Share of lubricating oil sent to incineration at end of life', 'value'].values[0])
IncinerationHazardous = (EndOfLifeInputParameters.loc[EndOfLifeInputParameters['Parameter'] == 'Share of hazardous waste sent to incineration at end of life', 'value'].values[0])
IncinerationPEHD = (EndOfLifeInputParameters.loc[EndOfLifeInputParameters['Parameter'] == 'Share of high-density polyethylene (HDPE) sent to incineration at end of life', 'value'].values[0])
IncinerationPEfoam = (EndOfLifeInputParameters.loc[EndOfLifeInputParameters['Parameter'] == 'Share of polyethylene foam sent to incineration at end of life', 'value'].values[0])
IncinerationPP = (EndOfLifeInputParameters.loc[EndOfLifeInputParameters['Parameter'] == 'Share of polypropylene (PP) sent to incineration at end of life', 'value'].values[0])
IncinerationEPS = (EndOfLifeInputParameters.loc[EndOfLifeInputParameters['Parameter'] == 'Share of expanded polystyrene (EPS) sent to incineration at end of life', 'value'].values[0])
IncinerationCardboard = (EndOfLifeInputParameters.loc[EndOfLifeInputParameters['Parameter'] == 'Share of cardboard sent to incineration at end of life', 'value'].values[0])
IncinerationPUfoam = (EndOfLifeInputParameters.loc[EndOfLifeInputParameters['Parameter'] == 'Share of polyurethane foam sent to incineration at end of life', 'value'].values[0])

#Materials that goes recycling
RecyclingMetals2 = (EndOfLifeInputParameters.loc[EndOfLifeInputParameters['Parameter'] == 'Share of special metals (e.g. stainless steel, chromium steel) sent to recycling at end of life', 'value'].values[0])
RecyclingMetals = (EndOfLifeInputParameters.loc[EndOfLifeInputParameters['Parameter'] == 'Share of metals (e.g low-alloyed steel) sent to recycling at end of life', 'value'].values[0])
RecyclingPEHD = (EndOfLifeInputParameters.loc[EndOfLifeInputParameters['Parameter'] == 'Share of high-density polyethylene (HDPE) sent to recycling at end of life', 'value'].values[0])
RecyclingConcrete = (EndOfLifeInputParameters.loc[EndOfLifeInputParameters['Parameter'] == 'Share of concrete sent to recycling at end of life', 'value'].values[0])
RecyclingClaybrick = (EndOfLifeInputParameters.loc[EndOfLifeInputParameters['Parameter'] == 'Share of clay bricks sent to recycling at end of life', 'value'].values[0])
RecyclingCardboard = (EndOfLifeInputParameters.loc[EndOfLifeInputParameters['Parameter'] == 'Share of cardboard sent to recycling at end of life', 'value'].values[0])

# Recycling efficiency process
RecyclingEfficiencyMetals = (EndOfLifeInputParameters.loc[EndOfLifeInputParameters['Parameter'] == 'Recycling efficiency of metals', 'value'].values[0])
RecyclingEfficiencyPlastics = (EndOfLifeInputParameters.loc[EndOfLifeInputParameters['Parameter'] == 'Recycling efficiency of plastics', 'value'].values[0])
RecyclingEfficiencyConcrete = (EndOfLifeInputParameters.loc[EndOfLifeInputParameters['Parameter'] == 'Recycling efficiency of concrete', 'value'].values[0])
RecyclingEfficiencyGlass = (EndOfLifeInputParameters.loc[EndOfLifeInputParameters['Parameter'] == 'Recycling efficiency of glass', 'value'].values[0])
RecyclingEfficiencyClaybrick = (EndOfLifeInputParameters.loc[EndOfLifeInputParameters['Parameter'] == 'Recycling efficiency of clay bricks', 'value'].values[0])
RecyclingEfficiencyMineralWool = (EndOfLifeInputParameters.loc[EndOfLifeInputParameters['Parameter'] == 'Recycling efficiency of mineral wool', 'value'].values[0])
RecyclingEfficiencyCardboard = (EndOfLifeInputParameters.loc[EndOfLifeInputParameters['Parameter'] == 'Recycling efficiency of cardboard', 'value'].values[0])

#Recycled content materials
RecycledContentMetals = (EndOfLifeInputParameters.loc[EndOfLifeInputParameters['Parameter'] == 'Recycled content of steel (fraction of total steel mass)', 'value'].values[0])
RecycledContentPlastic = (EndOfLifeInputParameters.loc[EndOfLifeInputParameters['Parameter'] == 'Recycled content of plastics (fraction of total plastic mass)', 'value'].values[0])
RecycledContentClaybrick = (EndOfLifeInputParameters.loc[EndOfLifeInputParameters['Parameter'] == 'Recycled content of clay bricks', 'value'].values[0])
RecycledContentConcrete = (EndOfLifeInputParameters.loc[EndOfLifeInputParameters['Parameter'] == 'Recycled content of concrete', 'value'].values[0])
RecycledContentCardboard = (EndOfLifeInputParameters.loc[EndOfLifeInputParameters['Parameter'] == 'Recycled content of cardboard', 'value'].values[0])
RecycledContentMineralWool = (EndOfLifeInputParameters.loc[EndOfLifeInputParameters['Parameter'] == 'Recycled content of mineral wool', 'value'].values[0])
RecycledContentGlass = (EndOfLifeInputParameters.loc[EndOfLifeInputParameters['Parameter'] == 'Recycled content of glass', 'value'].values[0])


#Substitution ration materials
SRmetals = (EndOfLifeInputParameters.loc[EndOfLifeInputParameters['Parameter'] == 'Material substitution ratio for metals', 'value'].values[0])
SRinert = (EndOfLifeInputParameters.loc[EndOfLifeInputParameters['Parameter'] == 'Material substitution ratio for inert materials', 'value'].values[0])
SRclaybrick = (EndOfLifeInputParameters.loc[EndOfLifeInputParameters['Parameter'] == 'Material substitution ratio for lightweight bricks', 'value'].values[0])

#Lower Heating Values materials for End-of-Life
LHVElastomer = (EndOfLifeInputParameters.loc[EndOfLifeInputParameters['Parameter'] == 'Lower heating value of elastomers', 'value'].values[0])
LHVElectronics = (EndOfLifeInputParameters.loc[EndOfLifeInputParameters['Parameter'] == 'Lower heating value of electronic waste', 'value'].values[0])
LHVHazardous = (EndOfLifeInputParameters.loc[EndOfLifeInputParameters['Parameter'] == 'Lower heating value of hazardous waste', 'value'].values[0])
LHVHDPE = (EndOfLifeInputParameters.loc[EndOfLifeInputParameters['Parameter'] == 'Lower heating value of high-density polyethylene (HDPE)', 'value'].values[0])
LHVPEfoam = (EndOfLifeInputParameters.loc[EndOfLifeInputParameters['Parameter'] == 'Lower heating value of polyethylene foam', 'value'].values[0])
LHVPP = (EndOfLifeInputParameters.loc[EndOfLifeInputParameters['Parameter'] == 'Lower heating value of polypropylene (PP)', 'value'].values[0])
LHVPVC = (EndOfLifeInputParameters.loc[EndOfLifeInputParameters['Parameter'] == 'Lower heating value of PVC', 'value'].values[0])
LHVEPS = (EndOfLifeInputParameters.loc[EndOfLifeInputParameters['Parameter'] == 'Lower heating value of expanded polystyrene (EPS)', 'value'].values[0])
LHVCardboard = (EndOfLifeInputParameters.loc[EndOfLifeInputParameters['Parameter'] == 'Lower heating value of cardboard', 'value'].values[0])
LHVPUfoam = (EndOfLifeInputParameters.loc[EndOfLifeInputParameters['Parameter'] == 'Lower heating value of polyurethane foam', 'value'].values[0])
LHVglass = (EndOfLifeInputParameters.loc[EndOfLifeInputParameters['Parameter'] == 'Lower heating value of glass wool', 'value'].values[0])
LHVFoamGlass = (EndOfLifeInputParameters.loc[EndOfLifeInputParameters['Parameter'] == 'Lower heating value of foam glass', 'value'].values[0])

# Energy recovery efficiency
Xincheat = (EndOfLifeInputParameters.loc[EndOfLifeInputParameters['Parameter'] == 'Thermal energy recovery efficiency', 'value'].values[0]) 
Xincele = (EndOfLifeInputParameters.loc[EndOfLifeInputParameters['Parameter'] == 'Electrical energy recovery efficiency', 'value'].values[0]) 